# Creating synthetic seismograms from a point source

**Table of Contents:**

* How to calculate synthetic seismograms from a point source

https://pyrocko.org/docs/current/library/examples/gf_forward.html#calculate-synthetic-seismograms-from-a-local-gf-store

From the previous module you have learned about the different source types and how to calculate and download a GF store. Now, we can proceed to calculate synthetic seismograms defining a source model. Let's try a double couple point source mechanism. 

**Importing modules and packages**

In [45]:
import os

from pyrocko.gf import LocalEngine, Target, DCSource, ws
from pyrocko import trace
from pyrocko.gui.marker import PhaseMarker

**Green's functions**

We need to download a GF store to be used.

In [54]:
# The store we are going extract data from.
store_id = 'iceland_reg_v2'

# First, download a Greens Functions store. If you already have one that you
# would like to use, you can skip this step and point the *store_superdirs* in
# the next step to that directory.

if not os.path.exists(store_id):
    ws.download_gf_store(site='kinherd', store_id=store_id)
    
# We need a pyrocko.gf.Engine object which provides us with the traces
# extracted from the store. In this case we are going to use a local
# engine since we are going to query a local store.
engine = LocalEngine(store_superdirs=['.'])    

**Target configuration**

Here we define the recording devices. This example refers to one station with a sensor of 3-components.

In [47]:
# Define a list of pyrocko.gf.Target objects.
channel_codes = 'ENZ'
targets = [Target(lat=10., lon=10., store_id=store_id, codes=('', 'STA', '', channel_code))
                  for channel_code in channel_codes]

**Source model definition**

Here we indicate the source model to be considered. In Pyrocko you can find the sources:
* Source
* SourceWithMagnitude
* SourceWithDerivedMagnitude
* ExplosionSource
* RectangularExplosionSource
* DCSource
* CLVDSource
* VLVDSource
* MTSource
* RectangularSource
* PseudoDynamicRupture
* DoubleDCSource
* RingfaultSource
* CombiSource
* SFSource
* PorePressurePointSource
* PorePressureLineSource

If you want to explore the code details of each source type you can go through https://pyrocko.org/docs/current/_modules/pyrocko/gf/seismosizer.html#Source.

For this example we will use the double couple source representation. The parameters needed for this source definition are:

In [48]:
source_dc = DCSource(
    lat=11.,
    lon=11.,
    depth=10000.,
    strike=0.,
    dip=40.,
    rake=90.,
    magnitude=6.)

After defining the source and targets, the data can be processed:

In [49]:
# This return a pyrocko.gf.Reponse object.
response = engine.process(source_dc, targets)

Now we can get the requested traces:

In [50]:
synthetic_traces = response.pyrocko_traces()

**Visualization with phase arrival picks**

To extract interpolated travel times of the phases defined in the GF store's config file, you can do:

In [51]:
store = engine.get_store(store_id)

markers = []
for t in targets:
    dist = t.distance_to(source_dc)
    depth = source_dc.depth
    arrival_time = store.t('begin', (depth, dist))
    m = PhaseMarker(tmin=arrival_time,
                    tmax=arrival_time,
                    phasename='P',
                    nslc_ids=(t.codes,))
    markers.append(m)

We can use Snuffler to visualize those traces together with the phase arrivals:

In [53]:
trace.snuffle(synthetic_traces, markers=markers)

cc.py:pyrocko.gui.snuffling     - ERROR    - Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pyrocko/gui/snuffling.py", line 1824, in load_if_needed
    self._module = __import__(self._name)
  File "/home/carrillo/.snufflings/ahfullgreen.py", line 8, in <module>
    from pyrocko.ahfullgreen import add_seismogram, Gauss, Impulse
ImportError: cannot import name 'Gauss'

cc.py:pyrocko.gui.pile_viewer   - WARNING  - Snuffling module "/home/carrillo/.snufflings/ahfullgreen.py" is broken
cc.py:pyrocko.gui.snuffling.okada - WARNING  - 
 /home/carrillo/.snufflings/okada/libokada.so: cannot open shared object file: No such file or directory
--> run 'make' in okada snuffling directory <--
